### Pong

Run this notebook and see what happens. 

Click the game canvas once it shows up to bring it into focus and then use the arrows and keys A and D to control the pedals.

* pong sound is by [freesound](https://freesound.org/people/NoiseCollector/sounds/4359/).
* Commodore 64 font is by [KreativeKorp](https://www.kreativekorp.com/software/fonts/c64.shtml).

In [1]:
import pyglet
import math
import time
import sys
import os

import PIL.Image

import numpy as np

In [2]:
p0 = os.path.abspath('.')
p1 = os.path.abspath(os.path.join(p0, '..'))

sys.path.insert(0, p1)

In [3]:
import jupylet.color

from jupylet.app import App, State
from jupylet.label import Label
from jupylet.sprite import Sprite

In [4]:
import pyglet.window.key as key

In [5]:
app = App(mode='jupyter', quality=20)

In [6]:
window = app.window

In [7]:
WIDTH = app.width
HEIGHT = app.height

In [8]:
background = '#3e32a2'
foreground = '#7c71da'

In [9]:
app.set_window_color(foreground)

In [10]:
a0 = np.ones((32, 32)) * 255
a1 = np.ones((128, 16)) * 255
a2 = np.ones((HEIGHT * 9 // 10, WIDTH * 9 // 10, 3)) * jupylet.color.color2rgb(background)[:3]

In [11]:
ball = Sprite(a0, y=HEIGHT/2, x=WIDTH/2, autocrop=True)

padl = Sprite(a1, y=HEIGHT/2, x=48)
padr = Sprite(a1, y=HEIGHT/2, x=WIDTH-48)

field = Sprite(a2, y=HEIGHT/2, x=WIDTH/2) 

In [12]:
sound = pyglet.media.load('sounds/pong-blip.wav', streaming=False)

In [13]:
pyglet.font.add_file('fonts/PetMe64.ttf')

In [14]:
scorel = Label(
    '0', font_name='Pet Me 64', font_size=42, color=foreground, 
    x=64, y=HEIGHT/2, anchor_y='center', anchor_x='left'
)

scorer = Label(
    '0', font_name='Pet Me 64', font_size=42, color=foreground, 
    x=WIDTH-64, y=HEIGHT/2, anchor_y='center', anchor_x='right'
)

In [15]:
@app.event
def on_draw():
    
    window.clear()
    
    field.draw()
    
    scorel.draw()
    scorer.draw()
    
    ball.draw()
    padl.draw()
    padr.draw()

In [16]:
state = State(
    
    sl = 0,
    sr = 0,
    
    bvx = 192,
    bvy = 192,
    
    vyl = 0,
    pyl = HEIGHT/2,

    vyr = 0,
    pyr = HEIGHT/2,

    left = False,
    right = False,

    key_a = False,
    key_d = False,
)

In [17]:
@app.event
def on_key_press(symbol, modifiers):
        
    if symbol == key.LEFT:
        state.left = True
        
    if symbol == key.RIGHT:
        state.right = True
        
    if symbol == key.A:
        state.key_a = True
        
    if symbol == key.D:
        state.key_d = True
        

@app.event
def on_key_release(symbol, modifiers):
    
    if symbol == key.LEFT:
        state.left = False
        
    if symbol == key.RIGHT:
        state.right = False

    if symbol == key.A:
        state.key_a = False
        
    if symbol == key.D:
        state.key_d = False
        

@app.run_me_again_and_again(1/120)
def update_pads(dt):
        
    if state.right:
        state.pyr = min(HEIGHT, state.pyr + dt * 512)
        
    if state.left:
        state.pyr = max(0, state.pyr - dt * 512)
        
    if state.key_a:
        state.pyl = min(HEIGHT, state.pyl + dt * 512)
        
    if state.key_d:
        state.pyl = max(0, state.pyl - dt * 512)
        
    ayl = 200 * (state.pyl - padl.y)
    ayr = 200 * (state.pyr - padr.y)

    state.vyl = state.vyl * 0.9 + (ayl * dt)
    state.vyr = state.vyr * 0.9 + (ayr * dt)
    
    padl.y += state.vyl * dt
    padr.y += state.vyr * dt
    
    padr.clip_position(WIDTH, HEIGHT)
    padl.clip_position(WIDTH, HEIGHT)

In [18]:
@app.run_me_again_and_again(1/120)
def update_ball(dt):
    
    bs0 = state.bvx ** 2 + state.bvy ** 2
    
    ball.rotation += 200 * dt
    
    ball.x += state.bvx * dt
    ball.y += state.bvy * dt
    
    if ball.top >= HEIGHT:
        app.play_once(sound)
        ball.y -= ball.top - HEIGHT
        state.bvy = -state.bvy
        
    if ball.bottom <= 0:
        app.play_once(sound)
        ball.y -= ball.bottom
        state.bvy = -state.bvy
        
    if ball.right >= WIDTH:
        app.play_once(sound)
        ball.x -= ball.right - WIDTH
        
        state.bvx = -192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sl += 1
        scorel.text = str(state.sl)
        
    if ball.left <= 0:
        app.play_once(sound)
        ball.x -= ball.left
        
        state.bvx = 192
        state.bvy = 192 * np.sign(state.bvy)
        bs0 = 0
        
        state.sr += 1
        scorer.text = str(state.sr)
        
    if state.bvx > 0 and ball.top >= padr.bottom and padr.top >= ball.bottom: 
        if 0 < ball.right - padr.left < 10:
            app.play_once(sound)
            ball.x -= ball.right - padr.left
            state.bvx = -state.bvx
            state.bvy += state.vyr / 2
            
    if state.bvx < 0 and ball.top >= padl.bottom and padl.top >= ball.bottom: 
        if 0 < padl.right - ball.left < 10:
            app.play_once(sound)
            ball.x += ball.left - padl.right
            state.bvx = -state.bvx
            state.bvy += state.vyl / 2
            
    bs1 = state.bvx ** 2 + state.bvy ** 2
    
    if bs1 < 0.9 * bs0:
        state.bvx = (bs0 - state.bvy ** 2) ** 0.5 * np.sign(state.bvx)

    ball.wrap_position(WIDTH, HEIGHT)

In [19]:
@app.run_me_now()
def highlights(dt):
    
    sl0 = state.sl
    sr0 = state.sr
    
    slc = np.array(scorel.color)
    src = np.array(scorer.color)
    
    while True:
        
        dt = yield 1/30
        
        r0 = 0.9 ** (120 * dt)
        
        scorel.color = np.array(scorel.color) * r0 + (1 - r0) * slc
        scorer.color = np.array(scorer.color) * r0 + (1 - r0) * src
        
        if sl0 != state.sl:
            sl0 = state.sl
            scorel.color = 'white'

        if sr0 != state.sr:
            sr0 = state.sr
            scorer.color = 'white'
            

In [20]:
app.run()

Canvas(height=512, width=512)

In [21]:
app.stop()

Ignoring call to stop() since it appears to have been done accidentally.